In [22]:
import bilby
from bilby import gw
from bilby.gw import detector as det
from bilby.gw import utils

psd_path = '/data/wiay/johnny/phd1/phd1/lib/python3.7/site-packages/bilby/gw/detector/noise_curves/aLIGO_ZERO_DET_high_P_psd.txt'
psd = det.psd.PowerSpectralDensity.from_power_spectral_density_file(psd_path)

snr = utils.optimal_snr_squared(signal=, power_spectral_density=psd, )

An example generating an SNR, although this isn't using bilby or pycbc, just doing ffts:

https://gilsay.physics.gla.ac.uk/gitlab/wei.changfeng/bilby_one_waveform_test/blob/master/snr_calculate.ipynb
